# CFP-FF data preparation notebook

The images are stored in a .bin file. They will be extracted and saved into an 
Images folder and a labels.csv file that stores the labels.

In [1]:
import pickle, os, cv2, io, pandas as pd, PIL.Image as Image
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
output_dir = os.path.join('..', 'data', 'datasets', 'face recognition', 'cfp_ff')
os.path.exists(output_dir)